# Basics of Jaxley

In this tutorial, we will introduce you to the basic concepts of Jaxley.
You will learn about:

- Modules
    - nodes
    - edges
- Channels
- Synapses
- Views
    - Groups

Here is a code snippet which you will learn to understand in this tutorial:
```python
import jaxley as jx
from jaxley.channels import Na, K, Leak
from jaxley.synapses import IonotropicSynapse
from jaxley.connect import connect
import matplotlib.pyplot as plt
import numpy as np


# Assembling different Modules into a Network
comp = jx.Compartment()
branch = jx.Branch(comp, nseg=1)
cell = jx.Cell(branch, parents=[-1, 0, 0])
net = jx.Network([cell]*3)

# Navigating and inspecting the Modules using Views
cell0 = net.cell(0)
cell0.nodes

# How to group together parts of Modules
net.cell(1).add_to_group("cell1")

# connecting two cells using a Synapse
pre_comp = cell0.branch(1).comp(0)
post_comp = net.cell1.branch(0).comp(0)

connect(pre_comp, post_comp)

# inserting channels in the membrane
with net.cell(0) as cell0:
    cell0.insert(Na())
    cell0.insert(K())

with net.cell(1) as cell1:
    cell0.insert(Leak())
```

First, we import the relevant libraries:

In [2]:
from jax import config
config.update("jax_enable_x64", True)
config.update("jax_platform_name", "cpu")

import jaxley as jx
from jaxley.channels import Na, K, Leak
from jaxley.synapses import IonotropicSynapse
from jaxley.connect import connect
import matplotlib.pyplot as plt
import numpy as np

# Modules

In Jaxley, we heavily rely on the concept of Modules to build biophyiscal models of neural systems at various scales.
Jaxley implements 4 Module types:
- `Compartment`
- `Branch`
- `Cell`
- `Network`

Modules can be connected together to build increasingly detailed and complex models. `Compartment` -> `Branch` -> `Cell` -> `Network`.

`Compartment`s are the atoms of biophysical models in Jaxley. All mechanisms and synaptic connections live on the level of `Compartment`s and can already be simulated using `jx.integrate` on their own. Everything you do in Jaxley starts with a `Compartment`.

In [3]:
comp = jx.Compartment() # single compartment model

Mutliple `Compartments` can be connected together to form longer, linear segments / cables, we call `Branch`es and are essentially equivalent to sections in `NEURON`.

In [4]:
nseg = 4
branch = jx.Branch([comp]*nseg)

In order to construct cell morphologies in Jaxley, multiple `Branches` can to be connected together using the `Cell` primitive.

In [5]:
parents = [-1,0,0] # soma = -1, since it has no parents and both dendrites connect to the soma (0). 
cell = jx.Cell([branch]*len(parents), parents)

Finally, several `Cell`s can be grouped together to form a `Network`, which can than be connected together using `Synpase`s.

In [10]:
ncells = 2
net = jx.Network([cell]*ncells)

net.shape # shows you the num_cells, num_branches, num_comps

(2, 6, 24)

`Module`s carry around the information about their current state and parameters in two Dataframes called `nodes` and `edges`.
`nodes` contains all the information that we associate with compartments in the model (each row corresponds to a compartment) and `edges` all the information relevant to synapses.

This means that you can easily keep track of the current state of your `Module` and how it changes at all times.

In [14]:
net.nodes

,local_cell_index,local_branch_index,local_comp_index,length,radius,axial_resistivity,capacitance,v,global_cell_index,global_branch_index,global_comp_index,controlled_by_param
0,0,0,0,10.0,1.0,5000.0,1.0,-70.0,0,0,0,0
1,0,0,1,10.0,1.0,5000.0,1.0,-70.0,0,0,1,0
2,0,0,2,10.0,1.0,5000.0,1.0,-70.0,0,0,2,0
3,0,0,3,10.0,1.0,5000.0,1.0,-70.0,0,0,3,0
4,0,1,0,10.0,1.0,5000.0,1.0,-70.0,0,1,4,0
5,0,1,1,10.0,1.0,5000.0,1.0,-70.0,0,1,5,0
6,0,1,2,10.0,1.0,5000.0,1.0,-70.0,0,1,6,0
7,0,1,3,10.0,1.0,5000.0,1.0,-70.0,0,1,7,0
8,0,2,0,10.0,1.0,5000.0,1.0,-70.0,0,2,8,0
9,0,2,1,10.0,1.0,5000.0,1.0,-70.0,0,2,9,0


In [9]:
net.edges.head() # this is currently empty since we have not made any connections yet

,global_edge_index,global_pre_comp_index,global_post_comp_index,pre_locs,post_locs,type,type_ind


# Views

Since these models can become arbitrarily complex, Jaxley utilizes so called `View`s to make working with `Modules` easy and intuitive. 

The simplest way to navigate Modules is by navigating them via the hierachy that we introduces above. Let's see how this works for a `Nework`.

In [16]:
net.cell(0) # View of the 0th cell of the network
net.cell(0).branch(0) # View of the 1st branch of the 0th cell of the network
net.cell(0).branch(1).comp(0) # View of the 0th comp of the 1st branch of the 0th cell of the network

# several types of indices are supported (lists, ranges, ...)
net.cell([0,1]).branch("all").comp(0) # View of all 0th comps of all branches of cell 0 and 1

branch.loc(0.1) # equivalent to `NEURON`s `loc`. Assumes branches are continous from 0-1.

net[0,0,0] # Modules/Views can also be lazily indexed

cell0 = net.cell(0) # views can be assigned to variables and only track the parts of the Module they belong to
cell0.branch(1).comp(0) # Views can be continuely indexed

View with 0 different channels. Use `.nodes` for details.

_In case you need even more flexibility in how you select parts of a Module, Jaxley provides a `select` method, to give full control over the exact parts of the `nodes` and `edges` that are part of a `View`. On examples of how this can be used, see [](MISSING)._

Views behave very similarly to `Module`s, i.e. the `cell0` (the 0th cell of the network) from the example above handles like the `cell` we instantiated earlier, which then became cell 0 of the network. As such `cell0` also has a `nodes` attribute, which keeps track of it's part of the network.

In [17]:
cell0.nodes

,local_cell_index,local_branch_index,local_comp_index,length,radius,axial_resistivity,capacitance,v,global_cell_index,global_branch_index,global_comp_index,controlled_by_param
0,0,0,0,10.0,1.0,5000.0,1.0,-70.0,0,0,0,0
1,0,0,1,10.0,1.0,5000.0,1.0,-70.0,0,0,1,0
2,0,0,2,10.0,1.0,5000.0,1.0,-70.0,0,0,2,0
3,0,0,3,10.0,1.0,5000.0,1.0,-70.0,0,0,3,0
4,0,1,0,10.0,1.0,5000.0,1.0,-70.0,0,1,4,0
5,0,1,1,10.0,1.0,5000.0,1.0,-70.0,0,1,5,0
6,0,1,2,10.0,1.0,5000.0,1.0,-70.0,0,1,6,0
7,0,1,3,10.0,1.0,5000.0,1.0,-70.0,0,1,7,0
8,0,2,0,10.0,1.0,5000.0,1.0,-70.0,0,2,8,0
9,0,2,1,10.0,1.0,5000.0,1.0,-70.0,0,2,9,0


Assigning `View`s to a variable makes it easuer to reuse parts of a `Module` later or to highlight them. However, this can become messy and we might need access to such a `View` more readily. For this purpose Jaxley implements so called groups that can be used to assign any `View` of a `Module` to an attribute, i.e. the soma.

In [22]:
net.cell("all").branch(0).comp(0).add_to_group("somas")
print("Groups", net.groups) # list the indices of the nodes dataframe that are part of the group

somas = net.somas # returns a View with only a subset of nodes
somas.nodes

Groups {'somas': array([ 0, 12])}


,local_cell_index,local_branch_index,local_comp_index,length,radius,axial_resistivity,capacitance,v,global_cell_index,global_branch_index,global_comp_index,controlled_by_param
0,0,0,0,10.0,1.0,5000.0,1.0,-70.0,0,0,0,0
12,1,0,0,10.0,1.0,5000.0,1.0,-70.0,1,3,12,0


In [ ]:
# connecting two cells using a Synapse
pre_comp = cell0.branch(1).comp(0)
post_comp = net.cell1.branch(0).comp(0)

connect(pre_comp, post_comp)